In [ ]:
# https://keras.io/
!pip install -q keras
import keras

In [ ]:
#Linking drive to colab to store datasets
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [ ]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Generate creds for the Drive FUSE library. Though the link asks you to verify twice, you don't have to!
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in Drive:')
!ls drive/

In [ ]:
# Create a file in a new directory called "Kaggle" in your Google Drive. This will be your operation base :P
#!echo "This newly created file will appear in your Drive file list. If you are reading this, that means the attempt to integrate was successful" > drive/EIP/created.txt
!cp drive/EIP/* .
!ls

In [ ]:
#For architecture
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

#For loading models
from pathlib import Path
from keras.models import load_model

#For data augmentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [ ]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [ ]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [ ]:
def add_transition(input, num_filter = 12, dropout_rate = 0.3):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [ ]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [ ]:
num_filter = 80
dropout_rate = 0.4
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (5,5), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [21]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 80)   6000        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 80)   320         conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 80)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [ ]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# checkpoint
from keras.callbacks import ModelCheckpoint
filepath= "weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
callbacks_list = [checkpoint]

In [15]:
#1-10
epoch = 0
e = 10
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 628s 13ms/step - loss: 1.5076 - acc: 0.4483 - val_loss: 1.7112 - val_acc: 0.4838

Epoch 00001: val_acc improved from -inf to 0.48380, saving model to weights-improvement-01-0.48.hdf5
Epoch 2/10
13952/50000 [=======>......................] - ETA: 6:54 - loss: 1.1552 - acc: 0.584350000/50000 [==============================] - 620s 12ms/step - loss: 1.0596 - acc: 0.6198 - val_loss: 1.9296 - val_acc: 0.5166

Epoch 00002: val_acc improved from 0.48380 to 0.51660, saving model to weights-improvement-02-0.52.hdf5
Epoch 3/10
29312/50000 [================>.............] - ETA: 3:57 - loss: 0.8666 - acc: 0.690250000/50000 [==============================] - 619s 12ms/step - loss: 0.8380 - acc: 0.7006 - val_loss: 1.0405 - val_acc: 0.6769

Epoch 00003: val_acc improved from 0.51660 to 0.67690, saving model to weights-improvement-03-0.68.hdf5
Epoch 4/10
33280/50000 [==================>........

MessageError: ignored

In [ ]:
!ls
#!cp *.hdf5 drive/EIP/
#!cp *.h5 drive/EIP

In [18]:
model.load_weights('weights-improvement-09-0.77.hdf5')

#10-20
epoch = 10
e = 20

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Train on 50000 samples, validate on 10000 samples
Epoch 11/20
50000/50000 [==============================] - 669s 13ms/step - loss: 0.4284 - acc: 0.8510 - val_loss: 1.0082 - val_acc: 0.7369

Epoch 00011: val_acc improved from -inf to 0.73690, saving model to weights-improvement-11-0.74.hdf5
Epoch 12/20
13696/50000 [=======>......................] - ETA: 7:10 - loss: 0.3869 - acc: 0.865750000/50000 [==============================] - 639s 13ms/step - loss: 0.3957 - acc: 0.8625 - val_loss: 0.7689 - val_acc: 0.7897

Epoch 00012: val_acc improved from 0.73690 to 0.78970, saving model to weights-improvement-12-0.79.hdf5
Epoch 13/20
29184/50000 [================>.............] - ETA: 4:06 - loss: 0.3551 - acc: 0.874950000/50000 [==============================] - 639s 13ms/step - loss: 0.3599 - acc: 0.8740 - val_loss: 0.7729 - val_acc: 0.7882

Epoch 00013: val_acc did not improve from 0.78970
Epoch 14/20
39808/50000 [======================>.......] - ETA: 2:00 - loss: 0.3377 - acc: 0.882350000

In [19]:
!ls
!cp *.hdf5 drive/EIP/
!cp *.h5 drive/EIP

datalab				  weights-improvement-07-0.76.hdf5
DNST_model.h5			  weights-improvement-09-0.77.hdf5
drive				  weights-improvement-11-0.74.hdf5
weights-improvement-01-0.48.hdf5  weights-improvement-12-0.79.hdf5
weights-improvement-02-0.52.hdf5  weights-improvement-14-0.82.hdf5
weights-improvement-03-0.68.hdf5  weights-improvement-20-0.82.hdf5
weights-improvement-06-0.71.hdf5


In [24]:
print('Using real-time data augmentation.')

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator( 
    rotation_range=90, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True) 

datagen.fit(x_train)

Using real-time data augmentation.


In [25]:
model.load_weights('weights-improvement-20-0.82.hdf5')

#20-25
epoch=20
e = 25
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

!ls
!cp *.hdf5 drive/EIP/
!cp *.h5 drive/EIP

Train on 50000 samples, validate on 10000 samples
Epoch 21/25
50000/50000 [==============================] - 670s 13ms/step - loss: 0.2766 - acc: 0.9024 - val_loss: 0.7347 - val_acc: 0.8164

Epoch 00021: val_acc improved from -inf to 0.81640, saving model to weights-improvement-21-0.82.hdf5
Epoch 22/25
13696/50000 [=======>......................] - ETA: 7:09 - loss: 0.2579 - acc: 0.908150000/50000 [==============================] - 636s 13ms/step - loss: 0.2602 - acc: 0.9077 - val_loss: 0.7440 - val_acc: 0.8308

Epoch 00022: val_acc improved from 0.81640 to 0.83080, saving model to weights-improvement-22-0.83.hdf5
Epoch 23/25
29184/50000 [================>.............] - ETA: 4:05 - loss: 0.2437 - acc: 0.911950000/50000 [==============================] - 635s 13ms/step - loss: 0.2516 - acc: 0.9092 - val_loss: 0.7689 - val_acc: 0.8195

Epoch 00023: val_acc did not improve from 0.83080
Epoch 24/25
39808/50000 [======================>.......] - ETA: 2:00 - loss: 0.2347 - acc: 0.917650000

In [ ]:
#25-35
epoch = 25
e = 35
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

!ls
!cp *.hdf5 drive/EIP/
!cp *.h5 drive/EIP

In [ ]:
!ls
!cp *.hdf5 drive/EIP/
!cp *.h5 drive/EIP

#40-50
epoch=40
e = 50
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

In [25]:
!ls
!cp *.hdf5 drive/EIP/
!cp *.h5 drive/EIP

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

10000/10000 [==============================] - 46s 5ms/step
Test loss: 1.2107041081428527
Test accuracy: 0.642


In [ ]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [4]:
model.load_weights('weights-improvement-20-0.82.hdf5')
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: ignored